In [1]:
#Setting up the simulation
#Imports
from zmqRemoteApi import RemoteAPIClient
import numpy
from matplotlib import pyplot as plt
import time

client = RemoteAPIClient()
sim = client.getObject('sim')
client.setStepping(True)
human = sim.getObject('./Bill')
tip = sim.getObject('./tip')


## The Human Walk Scenario
In this scenario a human walks towards the robot. After some time he stops walking, turns to a random direction and then continues walking in this direction

### Turning
I decided to use the 'Walking Bill' template from coppeliasim. I removed its object detection and made it just walk forwards for x seconds. When turning the orientation is set in radians. The used code is:
```
r[3]=r[3]+dt*2*speedModulator
```
with dt being the elapsed time in this step in seconds and speedModulator being 1.
#### Equal distribution
This means that for one full rotation, pi seconds have to elapse.
Using different distributions is very easy now, because we can just change the distribution of the turning time parameter.

In [2]:
#Using an equal distribution
def sendEqualParam():
    param = numpy.random.random() * numpy.pi * 0.25
    param += numpy.pi/4
    sim.setFloatSignal("rotationTime", param)

#### Using normal distribution
Using normal distribution is a bit more tricky. How much do we want Bill to deviate from simply walking forwards. I think standard devation should be a 90° or pi/4 turn.

In [3]:
#Using a normal distribution
def sendNormalParam():
    normaldist = numpy.random.normal(numpy.pi,numpy.pi/8)
    normaldist = numpy.abs(normaldist) + numpy.pi/4
    sim.setFloatSignal("rotationTime", normaldist)

In [4]:
def simulate():
    print("simulating")
    #send new parameters
    sendEqualParam()
    sim.setFloatSignal("robotSpeed", numpy.random.random() / 4 + 0.1)
    sim.boolparam_display_enabled = False
    sim.intparam_speedmodifier = 0.1
    
    min = 500

    sim.startSimulation()

    while (t := sim.getSimulationTime()) < 7:
        s = f'Simulation time: {t:.2f} [s]'
        
        result,distanceData,objs=sim.checkDistance(human,tip,0)
        dist = distanceData[6]
        if(dist<min):
            min = dist
        client.step()
    sim.stopSimulation()
    if min == 500:
        print("sus")
    return min

In [5]:
data = []

In [6]:
if(data == None):
    print("Instantiating data array")
    data = []
limit = 1.0
for i in range(0,100):
    min = simulate()
    print(min)
    data.append(min)
    if(min < limit):
        print("Critical!!!")
    time.sleep(0.2)


simulating
1.2611713409423828
simulating
1.2932050228118896
simulating
1.2932050228118896
simulating
1.2711069583892822
simulating
1.2822068929672241
simulating


KeyboardInterrupt: 

#### Using the scanner

The other component of the scenario is the laser scanner. To make the generated data points random I am using two different parameters for the scanner: Speed and distance accuracy.  